<a href="https://colab.research.google.com/github/akiitr/python/blob/master/fish_speech_tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fish Speech

In [ ]:
# cloning the repo
!git clone https://github.com/fishaudio/fish-speech

#base linux pkgs
!sudo apt-get install portaudio
!sudo apt-get install gcc
!sudo apt-get install portaudio19-dev python3-dev

# python env buildding
!pip install pip_tools
!pip-compile fish-speech/pyproject.toml --output-file requirements.txt
!pip install -r requirements.txt

#linux var
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

# change the version based on the latest released current 1.5
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

In [ ]:
# web gui (use public url as local does not work by passing app.launch(,share=True) in tools/run_webui.py)
# use compile for the Nvidia CUDA usage
# start the gui
!python fish-speech/tools/run_webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.5 \
    --decoder-checkpoint-path checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
     --compile

# Instruction from the repo:

### For Windows User / win用户

In [ ]:
!chcp 65001

### For Linux User / Linux 用户

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

### Prepare Model

In [ ]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com
!cd fish-speech/
!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir checkpoints/fish-speech-1.5/

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
!python fish-speech/tools/run_webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.5 \
    --decoder-checkpoint-path checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
    # --compile

## Break-down CLI Inference

### 1. Encode reference audio: / 从语音生成 prompt:

You should get a `fake.npy` file.

你应该能得到一个 `fake.npy` 文件.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\vo_hutao_draw_appear.wav"

!python tools/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text: / 从文本生成语义 token:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

> 该命令会在工作目录下创建 codes_N 文件, 其中 N 是从 0 开始的整数.

> 您可以使用 `--compile` 来融合 cuda 内核以实现更快的推理 (~30 tokens/秒 -> ~300 tokens/秒)

In [ ]:
!python tools/llama/generate.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.4" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens: / 从语义 token 生成人声:

In [ ]:
!python tools/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.4/firefly-gan-vq-fsq-8x1024-21hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)